# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

In [15]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# Iteration 1

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full'
section = 'discussion'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py

    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    return references_df_dict[iteration_id]
    # ###### 
    # # Step 2:  Add rows from gpt_queue table to sources table 
    # bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # # ##### 
    # # Step 4: Create summaries (functions contained in orm_summarize.py)
    # chatbot_dict = batch_summarize( # orm_summarize.py
    #     sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #     simplify_task, simplify_audience, format_task,
    #     chatbot_dict, temperature=temperature,
    #     system_role=system_role, model=model, max_tokens=1000,
    #     n_choices=n_choices, pause_per_request=pause_per_request,
    #     iteration_id=iteration_id, save_outputs=save_outputs
    #     )
    # #########
    # # Step 5: Create summaries table
    # qna_dict = create_summaries_df(
    #     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #     )

    # ##########
    # # Step 5: Add results to summaries and prompts table 
    # bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    # return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=False, article_limit=article_limit)
    qna_dict

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 1
Search term: (Effects of Exercise Alone or Combined With Cognitive Training and Vitamin D Supplementation to Improve Cognition in Adults With Mild Cognitive Impairment: A Randomized Clinical Trial [ti]) AND (JAMA Network Open [ta])
Check these PMIDs: ['37471089', '37368401', '36512354', '35604689', '35107574']
	Input title: effects of exercise alone or combined with cognitive training and vitamin d supplementation to improve cognition in adults with mild cognitive impairment: a randomized clinical trial
	Result title: effects of exercise alone or combined with cognitive training and vitamin d supplementation to improve cognition in adults with mild cognitive impairment a randomized clinical trial
	Cleaned input title: effects of exercise alone or combined with cognitive training and vitamin d supplementation to improve cognition in adults with mild cognitive impairment a randomized clinical trial
	Cleaned first result title: effect

# search_article

In [7]:
def search_article2(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                    if index == 0:
                        first_cleaned_result = cleaned_result
                        first_result_title = result_title
                        first_cleaned_result_title = cleaned_result_title
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {id_list[index]}.')
                    return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(index)
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {first_result_title if first_result_title else first_cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned first result title: {first_cleaned_result_title}\n') 
                result = retrieve_citation(id_list[0], api_key).decode('utf-8')
            return [cleaned_title, first_cleaned_result_title]     
    except Exception as error: 
        print(f'Response: \n{data}')
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    


api_key = os.getenv('api_ncbi') # Pubmed API key
title = "Effects of Exercise Alone or Combined With Cognitive Training and Vitamin D Supplementation to Improve Cognition in Adults With Mild Cognitive Impairment: A Randomized Clinical Trial"
publication = "JAMA Netw Open"


result = search_article2(title, publication, api_key, verbose=False)
result

Search term: (Effects of Exercise Alone or Combined With Cognitive Training and Vitamin D Supplementation to Improve Cognition in Adults With Mild Cognitive Impairment: A Randomized Clinical Trial [ti]) AND (JAMA Netw Open [ta])


'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2023//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_230101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">37471089</PMID><DateCompleted><Year>2023</Year><Month>07</Month><Day>21</Day></DateCompleted><DateRevised><Year>2023</Year><Month>07</Month><Day>23</Day></DateRevised><Article PubModel="Electronic"><Journal><ISSN IssnType="Electronic">2574-3805</ISSN><JournalIssue CitedMedium="Internet"><Volume>6</Volume><Issue>7</Issue><PubDate><Year>2023</Year><Month>Jul</Month><Day>03</Day></PubDate></JournalIssue><Title>JAMA network open</Title><ISOAbbreviation>JAMA Netw Open</ISOAbbreviation></Journal><ArticleTitle>Effects of Exercise Alone or Combined With Cognitive Training and Vitamin D Supplementation to Improve Cognition in Adults With Mild Cognitive Impairment: A Randomized Clinical Trial.</ArticleTi

In [4]:
result[0] == result[1]

True

# iteration 2

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full'
section = 'discussion'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py

    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    return references_df_dict[iteration_id]
    # ###### 
    # # Step 2:  Add rows from gpt_queue table to sources table 
    # bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # # ##### 
    # # Step 4: Create summaries (functions contained in orm_summarize.py)
    # chatbot_dict = batch_summarize( # orm_summarize.py
    #     sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #     simplify_task, simplify_audience, format_task,
    #     chatbot_dict, temperature=temperature,
    #     system_role=system_role, model=model, max_tokens=1000,
    #     n_choices=n_choices, pause_per_request=pause_per_request,
    #     iteration_id=iteration_id, save_outputs=save_outputs
    #     )
    # #########
    # # Step 5: Create summaries table
    # qna_dict = create_summaries_df(
    #     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #     )

    # ##########
    # # Step 5: Add results to summaries and prompts table 
    # bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    # return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=False, article_limit=article_limit)
    qna_dict

Query: SELECT * from gpt_queue ORDER BY id ASC LIMIT 1
Search term: (Effects of Exercise Alone or Combined With Cognitive Training and Vitamin D Supplementation to Improve Cognition in Adults With Mild Cognitive Impairment: A Randomized Clinical Trial [ti]) AND (JAMA Network Open [ta])
Number of abstract sections: 8


# *End of Page*